In [1]:
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from experiments.utils.get_avg_classwise_f1 import get_avg_classwise_f1
from sklearn.cluster import KMeans
import numpy as np
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle

In [2]:
rna = DataFetcher.fetch_data('rna N3k')
coli20 =  DataFetcher.fetch_data('coil20')
globe = DataFetcher.fetch_data('globe', size=7000)
fmnist = DataFetcher.fetch_data('fashion mnist', size=7000)
mnist = DataFetcher.fetch_data('mnist', size=7000)

datasets = [rna, coli20,globe,fmnist, mnist]
dataset_names = ['rna N3k', 'coli20', 'globe', 'fashion mnist', 'mnist']
termination_values = [0.0129,0.015,0.0019,0.01128,0.0218]


####################
Fetching the "rna N3k" dataset
####################
Dataset loaded
Dataset shape: (3000, 50)
####################
####################
Fetching the "coil20" dataset
####################
Dataset loaded
Dataset shape: (1440, 1024)
####################
####################
Fetching the "globe" dataset
####################
Dataset loaded
Dataset shape: (7003, 3)
####################
####################
Fetching the "fashion mnist" dataset
####################
Dataset loaded
Dataset shape: (7000, 784)
####################
####################
Fetching the "mnist" dataset


C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


####################
Dataset loaded
Dataset shape: (7000, 784)
####################


In [3]:
layouts = {"nesterov": {name : [] for name in dataset_names}, "no nesterov": {name : [] for name in dataset_names}}


In [4]:
num_repeats = 20
metric_collection = {"Average n-tet stress": 1}

for j,dataset in enumerate(datasets):
    Xld = PCA(n_components=2, whiten=False, copy=True).fit_transform(dataset.data).astype(np.float64)
    Xld *= 10/np.std(Xld)

    for i in range(num_repeats):
        squad = SNeD(dataset=dataset, initial_layout=Xld, use_nesterovs_momentum=True, ntet_size=4)
        layout_squad = LayoutCreation.create_layout(squad, terminate_at = termination_values[j],optional_metric_collection=metric_collection)
        layouts['nesterov'][dataset_names[j]].append(layout_squad)

        squad2 = SNeD(dataset=dataset, initial_layout=Xld, use_nesterovs_momentum=False, ntet_size=4)
        layout_squad2 = LayoutCreation.create_layout(squad2, terminate_at = termination_values[j],
                                                     optional_metric_collection=metric_collection)
        layouts['no nesterov'][dataset_names[j]].append(layout_squad2)



output_dir= (Path(PROJECT_ROOT).joinpath(
    Path(f"experiments/Nesterovs_momentum/out/"))).resolve().absolute()

path_to_pickle = (Path(output_dir).joinpath(Path(f"layouts.pickle"))).resolve()
with open(path_to_pickle, 'wb') as pickle_out:
    pickle.dump(layouts, pickle_out)


####################
A 2D layout of the "rna N3k" dataset will be created 
using the "Stochastic N-tet Descent MDS" algorithm
####################
The HD distance measure used is: euclidean
####################

 Nesterov's momentum will be used by the algorithm 

 "N-tet" size: 4
####################
"Average n-tet stress" will be measured on every iteration
####################
All stress calculations will be performed using the euclidian norm
####################
Iteration number: 0, Average n-tet stress 0
Iteration number: 1, Average n-tet stress 0.03127451724766348
Iteration number: 2, Average n-tet stress 0.02141015907626424
Iteration number: 3, Average n-tet stress 0.02006237963697885
Iteration number: 4, Average n-tet stress 0.019660702122682738
Iteration number: 5, Average n-tet stress 0.018685530232995098
Iteration number: 6, Average n-tet stress 0.019204285558826542
Iteration number: 7, Average n-tet stress 0.019196428468748236
Iteration number: 8, Average n-tet stress 0.019

KeyboardInterrupt: 